In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSN%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFDSN%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSN%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSN%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = fluxF.curr[f2s]
        # update solid flux interface
        solverS.update_flux_interface(interface_tag, fluxS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = tempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=6.
step=1, pc_iterations=4.
step=2, pc_iterations=4.
step=3, pc_iterations=4.
step=4, pc_iterations=4.
step=5, pc_iterations=4.
step=6, pc_iterations=4.
step=7, pc_iterations=4.
step=8, pc_iterations=4.
step=9, pc_iterations=4.
step=10, pc_iterations=4.
step=11, pc_iterations=4.
step=12, pc_iterations=4.
step=13, pc_iterations=4.
step=14, pc_iterations=4.
step=15, pc_iterations=4.
step=16, pc_iterations=4.
step=17, pc_iterations=4.
step=18, pc_iterations=4.
step=19, pc_iterations=4.
step=20, pc_iterations=4.
step=21, pc_iterations=4.
step=22, pc_iterations=4.
step=23, pc_iterations=4.
step=24, pc_iterations=4.
step=25, pc_iterations=4.
step=26, pc_iterations=4.
step=27, pc_iterations=3.
step=28, pc_iterations=4.
step=29, pc_iterations=4.
step=30, pc_iterations=3.
step=31, pc_iterations=4.
step=32, pc_iterations=4.
step=33, pc_iterations=3.
step=34, pc_iterations=4.
step=35, pc_iterations=3.
step=36, pc_iterations=4.
step=37, pc_iterations=3.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSN%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.88468771 0.89298029 0.89885438 0.90358998 0.90755432 0.91086009
 0.91375707 0.91630697 0.91860288 0.92072774 0.92263735 0.92440966
 0.92603711 0.92755364 0.92895473 0.93026376 0.93148122 0.93262702
 0.93369459 0.93469132 0.93562615 0.93650041 0.93732336 0.93809132
 0.93881041 0.93948264 0.9401162  0.94070742 0.9412631  0.9417824
 0.94226663 0.94272104 0.94314421 0.94353887 0.9439093  0.94425206
 0.94457125 0.94486652 0.94514222 0.94539607 0.94563065 0.94584758
 0.94604547 0.94622651 0.94639125 0.94654015 0.94667414 0.94679335
 0.94689828 0.94699046 0.94706819 0.94713356 0.9471855  0.94722487
 0.94725217 0.9472661  0.94726898 0.94725845 0.94723585 0.9472004
 0.94715208 0.9470917  0.94701695 0.94692913 0.94682581 0.94670879
 0.94657677 0.94642765 0.94626298 0.94607995 0.94587797 0.9456576
 0.94541503 0.94515235 0.94486484 0.94455283 0.94421133 0.94384309
 0.94344364 0.94300845 0.94253886 0.94202617 0.9414703  0.94086999
 0.94021345 0.9394999  0.93871788 0.93786576 0.93692892 0.9358930